# About: CoursewareHubインベントリの準備

CoursewareHubに必要なソフトウェアの設定は [Ansible](https://www.ansible.com/) を利用します。

これから設定するCoursewareHubに関する各種設定を、インベントリ ... 特に [Ansibleグループ変数](https://docs.ansible.com/ansible/latest/user_guide/playbooks_variables.html) として定義します。

全ノードのインベントリファイルが`./hosts`に存在していることを前提とします。

以下のようなフォルダ、ファイルにAnsible関連ファイルを配置します。

- Inventoryファイルは `./(指定したインベントリ名)`
- グループ変数定義ディレクトリは `./group_vars` とします。

# 準備

ファイルの準備をします。

In [ ]:
import os

group_vars_dir = './group_vars'
if not os.path.exists(group_vars_dir):
    os.mkdir(group_vars_dir)
!ls -la {group_vars_dir}

# 利用可能なサーバーの確認

設定対象のサーバーは、すでに `D01_CoursewareHub用VMをAWSに作成.ipynb` などで、Inventory(`./hosts`)に設定されているものとします。

In [ ]:
hosts_file = './hosts'
!cat {hosts_file}

Ansible Inventory中に定義されているグループ( `[グループ名]` のように`[]`で囲まれた定義 )のうち、CoursewareHub構築に使うグループ名を指定します。

In [ ]:
target_group = 'cwhtest0001'

# 接続確認
%env ANSIBLE_INVENTORY={hosts_file}
!ansible -m ping {target_group}

# サーバーの役割の決定

以下の3つのサーバーに対して、役割を設定します。

In [ ]:
hosts_stdout = !ansible -m ping {target_group}
hosts = [line.split()[0] for line in hosts_stdout if 'SUCCESS' in line]
hosts

以下のコードセルに、NFSサーバーとして使うサーバーと、CoursewareHub masterとして使うサーバーのそれぞれのInventoryでのIPアドレスを記入してください。

設定例)

**NFSサーバーをCousewareHubのノードと分離する場合**

* `10.1.1.100` ... NFSサーバー
* `10.1.1.101` ... CoursewareHubノード1
* `10.1.1.102` ... CoursewareHubノード2

```
# NFSサーバーとして使うサーバー
nfs_server = '10.1.1.100'

# CoursewareHub masterとして使うサーバー ... public_ipがついているものである必要があります
master_server = '10.1.1.101'

# いずれにも指定されていないサーバーはnodeとして自動的に設定されます。
```

**NFSサーバーをCousewareHubのmasterノードと共有する場合**

* `10.1.1.101` ... CoursewareHubノード1 / NFSサーバー
* `10.1.1.102` ... CoursewareHubノード2


```
# NFSサーバーとして使うサーバー
nfs_server = '10.1.1.101'

# CoursewareHub masterとして使うサーバー ... public_ipがついているものである必要があります
master_server = '10.1.1.101'

# いずれにも指定されていないサーバーはnodeとして自動的に設定されます。
```


In [ ]:
# NFSサーバーとして使うサーバー
nfs_server = '10.1.1.162'

# CoursewareHub masterとして使うサーバー ... public_ipがついているものである必要
master_server = '10.1.1.162'

# いずれにも指定されていないサーバーはworker nodeとして自動的に設定されます。

指定されたIPアドレス(`nfs_server`, `master_server`)がインベントリ中にあるか検証します。

以下のセル実行に失敗する場合は、前のセルを選択後Freezeを解除 <i class='fa fa-fw fa-snowflake-o' style='color:gray;'></i> し、正しい値を再設定、実行してください。

In [ ]:
assert nfs_server in hosts, 'nfs_server の指定がinventory中に見つかりません'
assert master_server in hosts, 'master_server の指定がinventory中に見つかりません'

# 以下は master_server に public_ipプロパティが定義されていないとエラーとなる
!ansible -m debug -a 'msg={{{{ public_ip }}}}' {target_group} -l {master_server}

問題なければ変数 `worker_servers` を定義する。

In [ ]:
worker_servers = list(set(hosts) - {nfs_server, master_server})
worker_servers

In [ ]:
print('nfs_server: {}'.format(nfs_server))
print('master_server: {}'.format(master_server))
print('worker_nodes:')
for node in worker_servers:
    print('  {}'.format(node))

CoursewareHubの識別子を設定する

マスターノードの台数と同じ数、CoursewareHubを構築することができます。そのため、個別のCoursewareHubを区別するための識別子を設定しておきます。

インベントリのグループ名、NFSのエクスポートパス等で使用されます。

NFSサーバーを複数のCoursewareHubで共有する場合には、互いに異なる名前を設定する必要があります。

In [ ]:
hubname = 'cwhtest0001'

# 各種パラメータの決定

Ansibleのgroup_vars用のdictを定義

In [ ]:
group_vars = {}

## Private IPアドレスの設定

サーバーに関して、**NFSとDocker Swarmの通信を別のIPアドレスで実施する場合**は以下のように対応表を定義してください。
指定されない場合は、Inventoryで定義されたIPでNFSとDocker Swarmの通信を行います。

```
servicenet_ips = {
  'Inventoryで定義されたIP': 'NFSとDocker Swarmの通信のために使用したいIP'
}
```

In [ ]:
servicenet_ips = {
}

IPアドレスがリスト中にあるかどうかを検証する。失敗する場合は↑の対応表を修正する。

In [ ]:
for ip in servicenet_ips.keys():
    assert ip in ([master_server] + worker_servers + [nfs_server])

## NFSパラメータの決定

NFSサーバー上で、NFSボリュームとしてexportするパスを指定します。

In [ ]:
group_vars['nfspath'] = '/exported/{hubname}'.format(hubname=hubname)
group_vars['nfspath']

## CoursewareHubパラメータの決定

CoursewareHubのFQDNを設定する。

登録済みのドメイン名がある場合は、`group_vars['master_fqdn']`にそのドメイン名を設定すること。

In [ ]:
group_vars['master_fqdn'] = 'example.com'
group_vars['master_fqdn']

FQDNから、LTI認証連携用のendpointが決まります。

In [ ]:
print('Tool URL / Launch URL:\n\t https://{}/'.format(group_vars['master_fqdn']))
print('Initiate login URL / LTI 1.3 Tool OpenID Connect/Initialization Endpoint:\n\t https://{}/php/lti/login.php'.format(group_vars['master_fqdn']))
print('Redirection URI / LTI 1.3 Tool Redirect Endpoint:\n\t https://{}/php/lti/service.php'.format(group_vars['master_fqdn']))

### フェデレーションへの参加・不参加の設定

In [ ]:
group_vars['enable_federation'] = False
group_vars['enable_federation']

### フェデレーションに直接参加しないSP用の設定

* idp-proxyを使用して、間接的に学認を使用する
* ローカルユーザーのみを使用する

idp-proxyを利用する場合は、idp-proxyのFQDNを設定する。

idp-proxyを利用せず、ローカルユーザーのみを利用する場合は、空文字列を設定する。


In [ ]:
#group_vars['auth_fqdn'] = 'cwhidptest.example.nii.ac.jp'
group_vars['auth_fqdn'] = ''
group_vars['auth_fqdn']

### フェデレーションに直接参加するSP用の設定

idp-proxyを利用せずに、直接フェデレーションに参加する。

フェデレーションとクラウドゲートウェイの設定を行う。

テストフェデレーションの使用の有無を選択

In [ ]:
group_vars['enable_test_federation'] = False
group_vars['enable_test_federation']

DSのサーバー名

- 未設定ならデフォルト値が使用されます。
- テストフェデレーションを使用するなら、適切な値の設定が必要です。

In [ ]:
# group_vars['gakunin_ds_hostname'] = 'test-ds.gakunin.nii.ac.jp'
# group_vars['gakunin_ds_hostname']

クラウドゲートウェイサーバーの設定

- 未設定ならデフォルト値が使用されます。
- テストフェデレーションを使用するなら、適切な値の設定が必要です。

In [ ]:
# group_vars['cg_fqdn'] = 'sptest.cg.gakunin.jp'
# group_vars['cg_fqdn']

フェデレーションメタデータ検証用証明書の用意

In [ ]:
# group_vars['ds_metadata_signer_url'] = 'https://metadata.gakunin.nii.ac.jp/gakunin-test-signer-2011.cer'
# group_vars['ds_metadata_signer_url']

**テスト用クラウドゲートウェイを利用するときのみ**

クラウドゲートウェイメタデータの用意

学認から別途入手し、ローカルに配置済みとする。

In [ ]:
# group_vars['cgidp_metadata'] = 'sptestcgidp-metadata-self-signed-cert.xml.20200305'
# group_vars['cgidp_metadata']

In [ ]:
if 'cgidp_metadata' in group_vars:
    !ls -l {group_vars[cgidp_metadata]}

### クラウドゲートウェイグループの設定

使用を許可するクラウドゲートウェイグループのグループIDを列挙する。

idp-proxyを使用するか、フェデレーションに直接参加しているときのみ有効。

ローカルユーザーのみの場合は、空の配列を指定する。

In [ ]:
#cg_groups = [
#   'https://sptest.cg.gakunin.jp/gr/coursewarehub-test'
#]
group_vars['cg_groups'] = [
]
group_vars['cg_groups']

### Cullingの設定

アイドル状態のNotebookサーバーを自動的に停止する設定を行います

|項目                     | 説明                                                           |
|-------------------------|----------------------------------------------------------------|
|cull_server              | アイドル状態のNotebookサーバーの停止を有効／無効化 (yes/no/0/1)|
|cull_server_idle_timeout | アイドル状態のNotebookサーバーの停止までのタイムアウト時間     |
|cull_server_max_age      | アイドル状態でなくてもNotebookサーバーを停止するまでの時間     |
|cull_server_ every       | Notebookのアイドル状態のタイムアウトのチェック間隔             |


In [ ]:
group_vars['cull_server'] = 'no'
# group_vars['cull_server_idle_timeout'] = 43200

### JupyterHubのSpawnerの設定

JupyterHubがユーザーのNotebookサーバーを起動するさいのパラメータを設定します。


|項目                     | 説明                                                           |
|-------------------------|----------------------------------------------------------------|
|concurrent_spawn_limit   | ユーザーコンテナ同時起動処理数                                 |
|spawner_http_timeout     | ユーザーNotebookサービス 起動タイムアウト時間(秒)              |
|spawner_start_timeout | アイドル状態のNotebookサーバーの停止までのタイムアウト時間     |


In [ ]:
group_vars['concurrent_spawn_limit'] = 20
group_vars['spawner_http_timeout'] = 120
group_vars['spawner_start_timeout'] = 60

### Single-user Server Application

Classic Jupyter NotebookとJupyterLabを切り替えます

In [ ]:
group_vars['jupyterhub_singleuser_app'] = 'notebook.notebookapp.NotebookApp'   # Classic Notebook Server
# group_vars['jupyterhub_singleuser_app'] = 'jupyter_server.serverapp.ServerApp' # JupyterLab

### その他のパラメータ

通常は変更する必要はありません。

NFS構成設定

In [ ]:
nfs_server_dedicated = nfs_server != master_server
nfs_server_dedicated

In [ ]:
group_vars['nfsserver'] = nfs_server
group_vars['exchange'] = '/exchange/'
if nfs_server_dedicated:
    group_vars['nfs_server_role'] = 'nfs_server_dedicated'
    group_vars['nfsnodes_group'] = f'{hubname}_nodes'
else:
    group_vars['nfs_server_role'] = 'nfs_server'
    group_vars['nfsnodes_group'] = 'cwhub_nodes'

イメージ名指定

In [ ]:
group_vars['auth_proxy_image'] = 'niicloudoperation/coursewarehub-auth-proxy:master'
group_vars['jupyterhub_image'] = 'niicloudoperation/coursewarehub-jupyterhub:master'

# Ansibleリソースの出力

このNotebookで設定された内容は、AnsibleのInventory, Ansibleのgroup_varsに保存することで、この後のNotebook実行で利用することができます。

In [ ]:
nfs_server_with_ip = '{} servicenet_ip={}'.format(nfs_server, servicenet_ips[nfs_server] if nfs_server in servicenet_ips else nfs_server)
nfs_server_with_ip

In [ ]:
master_server_with_ip = '{} servicenet_ip={}'.format(master_server, servicenet_ips[master_server] if master_server in servicenet_ips else master_server)
master_server_with_ip

In [ ]:
worker_servers_with_ips = ['{} servicenet_ip={}'.format(node_server, servicenet_ips[node_server] if node_server in servicenet_ips else node_server)
                        for node_server in worker_servers]
worker_servers_with_ips

## Ansible Inventoryの出力

構築、操作対象のCoursewareHub用のインベントリを生成する。

インベントリファイルのパスを決定します。

ファイル名は、CoursewareHubの名前(`hubname`)を元に決定されます。

In [ ]:
import os.path

inventory_name = '{}_inventory'.format(hubname)

inventory_path = os.path.relpath(
    os.path.join(os.path.dirname(os.path.abspath(hosts_file)), inventory_name),
    start=os.getcwd())
inventory_path

In [ ]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

現在のinventoryがある場合は、後の比較のため、一時退避

In [ ]:
if os.path.exists(inventory_path):
    !cp -a {inventory_path} {work_dir}/inventory_old
else: 
    !touch {work_dir}/inventory_old

inventoryの生成

In [ ]:
worker_servers_list_with_ips = '\n'.join(worker_servers_with_ips)

with open(inventory_path, 'w') as f:
    f.write(f'''
[{hubname}:vars]
ansible_ssh_user=centos
ansible_ssh_private_key_file=~/.ssh/ansible_id_rsa

[{hubname}:children]
cwhub_nfs_server
cwhub_master
cwhub_nodes

[{hubname}_nodes:children]
cwhub_master
cwhub_nodes

[cwhub_nfs_server]
{nfs_server_with_ip}

[cwhub_master]
{master_server_with_ip}

[cwhub_nodes]
{worker_servers_list_with_ips}
'''.lstrip())
    
!cat {inventory_path}

変更箇所を確認

In [ ]:
!diff -u {work_dir}/inventory_old {inventory_path} ;:

## Ansible group_vars

In [ ]:
group_vars

group_varsを一時ファイルディレクトリに書き出す

In [ ]:
import yaml

with open(os.path.join(work_dir, hubname), 'w') as f:
    f.write(yaml.dump(group_vars))

In [ ]:
!cat {work_dir}/{hubname}

現在のgroup_varsと比較

In [ ]:
!diff -u {group_vars_dir}/{hubname} {work_dir}/{hubname} ;:

コピー

In [ ]:
!cp {work_dir}/{hubname} {group_vars_dir}/{hubname}

# 後始末

一時ファイルを削除する。

In [ ]:
!rm -fr {work_dir}